In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !unzip "/content/drive/MyDrive/Google Colab/HumanML3D.zip"

In [1]:
import os
from tqdm import tqdm
import torch
from transformers import pipeline

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
EXTERNAL_REPOS_DIR = os.path.join(ROOT_DIR, "external_repos")
MOMASK_REPO_DIR = os.path.join(EXTERNAL_REPOS_DIR, "momask-codes")
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"
# HUMAN_ML_DIR = os.path.join(ROOT_DIR, "content", "HumanML3D")
HUMAN_ML_DIR = os.path.join(MOMASK_REPO_DIR, "dataset", "HumanML3D")


print(f"Using device: {DEVICE}")

/opt/miniconda3/envs/digitalHumans/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: mps


In [2]:
texts_path = os.path.join(HUMAN_ML_DIR, "texts")
test_txt_path = os.path.join(HUMAN_ML_DIR, "test.txt")

# Load data
with open(test_txt_path, "r") as test_file:
    test_filenames = [line.strip() for line in test_file.readlines()]

def read_text_file(filepath):
    with open(filepath, "r") as file:
        return [line.strip().split('#')[0] for line in file.readlines()]

text_files_content = {}

# Iterate through the list of filenames and load the corresponding text files
for filename in test_filenames:
    file_path = os.path.join(texts_path, filename + ".txt")
    if os.path.exists(file_path):
        content = read_text_file(file_path)
        text_files_content[filename] = content
    else:
        print(f"File {filename} not found in {texts_path}")

print(f"Number of text files loaded: {len(text_files_content)}")

Number of text files loaded: 4384


In [18]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=DEVICE)

# Define your candidate labels
candidate_labels = ["dancing", "limbs", "emotions"]

# Define the number of top labels to consider
top_n = len(candidate_labels)
THRESHOLD = 0.87

dancing = {}
limbs = {}
emotions = {}

# Perform zero-shot classification in batches
for idx, (filename, lines) in enumerate(tqdm(text_files_content.items(), desc="Processing files: ")):
    for line in lines:
        result = classifier(line, candidate_labels)

        for k in range(top_n):
            label = result['labels'][k]
            score = result['scores'][k]
            if label == "dancing" and score >= THRESHOLD:
                if filename in dancing:
                    dancing[filename].append((line, score))
                else:
                    dancing[filename] = [(line, score)]
            elif label == "limbs" and score >= THRESHOLD:
                if filename in limbs:
                    limbs[filename].append((line, score))
                else:
                    limbs[filename] = [(line, score)]
            elif label == "emotions" and score >= THRESHOLD:
                if filename in emotions:
                    emotions[filename].append((line, score))
                else:
                    emotions[filename] = [(line, score)]

    if idx == 20:
        break

print("Zero-shot classification completed successfully!")

Processing files:   0%|          | 20/4384 [00:04<15:43,  4.63it/s]

Zero-shot classification completed successfully!


In [19]:
import json
beauty = json.dumps(limbs, indent=4)

print(beauty)

{
    "004822": [
        [
            "person walking at a average pace forward, swaying arms and torso with a sense of swagger",
            0.9761677980422974
        ]
    ],
    "014457": [
        [
            "a person stands, bent slightly forward, holds onto something with both hands and swings their arms as if they were hitting a golf ball.",
            0.9012690186500549
        ]
    ],
    "002530": [
        [
            "the person takes 4 steps forward starting with his right foot.",
            0.8770280480384827
        ]
    ],
    "004945": [
        [
            "person is bent down trying to pick up stuff, left arm is moved to the back, picks up more stuff and touches back again while bending down",
            0.9761883020401001
        ],
        [
            "a person, searching for something with their right hand, picks up the item with their left hand and places it in something by their head.",
            0.8705397844314575
        ]
    ],
    "001969

In [5]:
for filename, lines in emotions.items():
    print(f"Filename: {filename}")
    for line in lines:
        print(f"Line: {line[0]}")
        print(f"Score: {line[1]}")

    print()

In [22]:
not_limbs = set(test_filenames) - set(limbs.keys())
not_limbs = sorted(list(not_limbs))

print(f"Found {len(dancing)} files with 'dancing' label")
print(f"Found {len(limbs)} files with 'limbs' label")
print(f"Found {len(emotions)} files with 'emotions' label")
print(f"Files without 'limbs' label: {not_limbs}")

Found 122 files with 'dancing' label
Found 3511 files with 'limbs' label
Found 6 files with 'emotions' label


In [34]:
# Function to write results to a text file
def save_filenames(output_path, category):

    if isinstance(category, dict):
        with open(output_path, 'w') as f:
            for filename in category.keys():
                f.write(f"{filename}\n")
            f.write("\n")
    elif isinstance(category, (set, list)):
        with open(output_path, 'w') as f:
            for filename in category:
                f.write(f"{filename}\n")
            f.write("\n")

save_filenames("/content/dancing.txt", dancing)
save_filenames("/content/limbs.txt", limbs)
save_filenames("/content/not_limbs.txt", not_limbs)
save_filenames("/content/emotions.txt", emotions)

In [8]:
# Load not limbs
not_limbs = read_text_file("test_classes/not_limbs.txt")

# Load files with no limbs
for filename in not_limbs:
    file_path = os.path.join(texts_path, filename + ".txt")
    if os.path.exists(file_path):
        content = read_text_file(file_path)
        text_files_content[filename] = content
    else:
        print(f"File {filename} not found in {texts_path}")

# Print first 5 files with no limbs
for idx, (filename, lines) in enumerate(text_files_content.items()):
    print(f"Filename: {filename}")
    for line in lines:
        print(f"Line: {line}")
    print()

    if idx == 4:
        break

File  not found in /Users/maxkieffer/Documents/github/DigitalHumans/external_repos/momask-codes/dataset/HumanML3D/texts
Filename: 004822
Line: a person is walking in place at a slow pace.
Line: person walking at a average pace forward, swaying arms and torso with a sense of swagger
Line: a person walks in place.

Filename: 014457
Line: the person swings a golf club.
Line: a person swings a golf club.
Line: a person stands, bent slightly forward, holds onto something with both hands and swings their arms as if they were hitting a golf ball.

Filename: 009613
Line: the man runs back wards
Line: the person is running backwards quickly.
Line: a person jogs backwards, diagonally.

Filename: 008463
Line: a man walks forward, then squats to pick something up with both hands, stands back up, and resumes walking.
Line: walking forward and then bending down.
Line: man walks along, then bends down and picks something up.

Filename: 014160
Line: a person waves with his right hand.
Line: a person s